# Basic scraping for Guardian discussions u

In [49]:
! ./.g


[master 07063a7] web
 2 files changed, 706 insertions(+), 1060 deletions(-)
 create mode 100644 ej7f5.pkl
 rewrite guardian comments scraper.ipynb (82%)
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 105.96 KiB | 1.31 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/macbuse/GUARDIAN.git
   75ebeea..07063a7  master -> master


In [59]:
with open('guardian comments scraper.ipynb','r') as fp:
    nb = fp.read()

In [60]:
import re

pp = re.compile("(https://www.the.*?)'")
lks = pp.findall(nb)

with open('links.txt','a') as fp:
    fp.write('\n'.join(lks))

In [1]:
import re 
import requests
import pickle
import json

In [14]:
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/19/ditch-the-algorithm-generation-students-a-levels-politics'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/21/joe-biden-democrat-convention-nice-guy-trump-fight'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/23/the-cruel-exams-algorithm-has-laid-bare-the-unfairness-at-the-heart-of-our-schools'
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/01/us-mainstream-right-vigilante-terror'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/31/politicians-covid-19-statistics-statisticians'

In [21]:
key = my_keys.keys['guardian']

NameError: name 'my_keys' is not defined

In [15]:
#scrape the page and get the short url
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)

#the short url allows us to access the discussion
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 50
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn ,'wb') as fp:
    pickle.dump(json_data,fp)
    


getting comments page no. 1
there are 10 pages
getting comments page no. 2
getting comments page no. 3
getting comments page no. 4
getting comments page no. 5
getting comments page no. 6
getting comments page no. 7
getting comments page no. 8
getting comments page no. 9
getting comments page no. 10
archiving to eyhx8.pkl


In [2]:
json_data = pickle.load(open('dpb6e.pkl','rb') ) #jenkins

archiving to dqhaa.pkl #jenkins

archiving to evdjz.pkl #big cummings

e9j7e.pkl # bragg

eghyd.pkl # migrants

SyntaxError: invalid syntax (<ipython-input-2-7a525c39f9dc>, line 3)

In [3]:
json_data = pickle.load(open('etenk.pkl','rb') ) #jenkins


## remember each page of comments has been stored separately

In [4]:
dd = json.loads(json_data[0])
len(json_data)

8

In [8]:
xx = json.loads(json_data[0])
xx.keys()

dict_keys(['postedCommentHtml', 'commentsHtml', 'commentCount', 'refreshStatus', 'lastPage', 'paginationHtml'])

In [9]:
print( xx['commentsHtml'][:100])



<div class="d-discussion d-discussion--recommendations-open u-cf"
     data-read-only="false" >

 


In [16]:
pa = re.compile('data-comment-author="(.*?)"')
pid = re.compile('data-comment-author-id="(\d+)')

auths = []
for xx in json_data:
    hh = json.loads(xx)['commentsHtml']
    auths.extend(list( zip(pa.findall(hh) , pid.findall(hh)) ))

auths = list(set(auths))
auths.sort(key=lambda x : x[0].lower())


In [5]:
auths = dict(auths)

In [17]:
class Comment():
    pass

from bs4 import BeautifulSoup

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  
            soup.findAll('div', {'class' : "d-comment__body"}) ]
    
    metadata = soup.findAll('span', {'class' : "d-comment__author"})
    
    auth = [x.text.upper().strip() for x in metadata]
    auth_id = [ x.find('a')['href'] for x in metadata]
    
    return [ '##{}\n{}\n\n{}'.format(a,b,c.strip())  
                                 for a,b,c in zip(auth, auth_id, raw)]
     
all_texts = [ html2comments(page) for page in json_data[:] ]



from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

with open('b7jtb.pkl','rb') as fp:
    json_data = pickle.load(fp)

In [18]:
A = [x.split('\n')[0] for x in all_comments]
from collections import Counter
AN = Counter(A)

In [19]:
sorted([(x,y) for x,y in AN.items() if y > 4], key=lambda x : -x[1]) 

[('##SPIKE501', 34),
 ('##JDRMCR', 33),
 ('##PASTISANDCHIPS', 23),
 ('##BEADYFRY', 21),
 ('##CANARYATTHEWHARF', 19),
 ('##FIRMBELIEVER', 18),
 ('##JONBRADSHAW2020', 17),
 ('##COMMENTOR76', 15),
 ('##JACKDURHAM2020', 15),
 ('##ARTUSOV', 14),
 ('##DAVE6723', 14),
 ('##MARTINSILENUS', 12),
 ('##PUMUCKLMAN', 12),
 ('##HUGHBRYANT', 12),
 ('##HAGGALA', 12),
 ('##MILL1806', 11),
 ('##RATHERBEINOZ', 11),
 ('##EASTONHALL33', 11),
 ('##DISCUZ', 10),
 ('##RICHARDJWILLIAMS', 10),
 ('##BIELSA', 9),
 ('##DISABLEDSCAPEGOAT', 9),
 ('##ENGLISHATHENA', 8),
 ('##STECLOUD', 8),
 ('##PARTOFTHEPROBLEM', 8),
 ('##KINGOFTHEKOP', 7),
 ('##STEVEINBAVARIA', 7),
 ('##FISHPLATE', 7),
 ('##ATTILA9000', 7),
 ('##THEWHOFAN', 7),
 ('##AMORYC', 7),
 ('##ARCAIS', 6),
 ('##IFITSMAGIC', 6),
 ('##1ESSEX', 6),
 ('##SPORADICWIT', 6),
 ('##DIANAB', 6),
 ('##TEATOWL', 6),
 ('##AVBROWN', 6),
 ('##SECONDTHOUGHTS2', 6),
 ('##VERAANTHONY', 6),
 ('##SAGARMATHA1953', 6),
 ('##USEDTOLOVELEARNING', 5),
 ('##DARTINGKISS', 5),
 ('##CELT

In [20]:
all_comments.sort(key=lambda x : -len(x) )

In [21]:
print( '\n\n'.join(all_comments[:20]) )

##COGITAANTESALIS
https://profile.theguardian.com/user/id/12707619

When politicians cite Covid-19 statistics, they may be wrong – it doesn't mean the numbers are,
Well, I take two issue with the headline. First, politicians and media pundits are often cherry picking statistics to create narratives, frame certain topics and suggest solutions. If the UK Statistics Authority would have ensured that the wide public can access statistics that have been independently validated, maybe many toxic narratives would have been easier to debunk, wether the topic is our EU membership, taxes, trade and geopolitical relations, education, healthcare or immigration, or anything else.
If numbers would always be right, the way how they are aggregated would't be subject to arbitrary formulae and bias, then people would be in a better place to make informed choices. But they aren't.
Second, the recent public outrage in relation to numbers, formulae and computer modelling used to measure educational attainm

In [18]:
#https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gregmcshane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
a = '''My views haven't changed. Leaving the EU will be of no benefit to anyone except those who hold money and power along with certain corporates and a smidgen of medium to small businesses.
How else can it be when you are shutting shop to the most developed nations, the richest nations in the world, who are either in the EU or have market agreements with the EU. Even the emerging markets of China and India along with other developing economies are aspiring to develop trading agreements with the EU, the second largest economy in the world. Any deal with the UK will be secondary and will depend upon their agreement with the EU.
Boris Johnson once said "f**k business". I take this as meaning there is something in it for them and they don't really care if businesses find it difficulA
t to cope.
I can see ministers and conservative MPs queuing to fill their pockets by acting as advisors for potential trade deals whilst the rest of the country are left struggling trying to rebuild their lives, especially following the pandemic. The same ministers will probably be looking to lower all kinds of standards from food and welfare to safety at work to improve their opportunities.
When it comes to brinkmanship, I can only view this as getting away with whatever they can. I do not believe it is anything to do with negotiating an agreement that will be to the benefit of the UK or the EU as a whole, I don't believe our ministers are capable of doing that.'''

In [24]:
sid.polarity_scores(a)
sid.sentiment_valence(a)

TypeError: sentiment_valence() missing 4 required positional arguments: 'sentitext', 'item', 'i', and 'sentiments'

In [13]:
target = 'decolon'
target = 'science'
target = 'riot'


xx = [x for x in all_comments if target.lower() in x.lower() and  len(x) > 20]
yy = [x for x in all_comments if '#pin' in x.lower() and 2000 > len(x) > 20]

#print('\n\n'.join( ['\n\n'.join(x) for x in zip(xx,yy)])  )


#print( '\n\n'.join(xx).replace(target, ) )
pxx = re.compile(target, re.I)
print( pxx.sub('**' + target.upper() + '**', 
              '\n\n'.join(xx)) )
               
#print( '\n\n'.join(xx) )


##PARTOFTHEPROBLEM
https://profile.theguardian.com/user/id/16171854

It is alleged that having travelled from Illinois to Wisconsin to point his assault rifle at unarmed protesters, he shot two people dead.

That's one way of phrasing it I suppose, another would be that he lived 20-30 minutes away across the state border (Illinois) and worked in Kenosha... I thought he was there to protect a specific business (at the request of the owner, that's according to some reports) and it was to protect them from the **RIOT**ers (the ones setting the fires that have burned down buildings).

...the president himself defended Rittenhouse, saying he appeared to have been acting in self defence.

He might have been or do you think it's impossible? I can comfortably say that if he wasn't acting in self-defense then he should be prosecuted to the fullest extent of the law, but it doesn't look like you're happy with the idea that it might be justified.
Like it or not, not all the protesters are peacefu

36